In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

## Loan Status Classification

In [2]:
# ppp1 = pd.read_csv('../data/public_up_to_150k_1_230930.csv')
path = '../data/'
dir_list = os.listdir(path)[:-1] # Exclude the data dict
print(dir_list)

['public_up_to_150k_3_230930.csv', 'public_up_to_150k_11_230930.csv', 'cleaned_1and2_reduced.csv', 'public_up_to_150k_6_230930.csv', 'public_up_to_150k_1_230930.csv', 'public_up_to_150k_9_230930.csv', 'cleaned_ppp.csv', 'public_up_to_150k_4_230930.csv', 'public_up_to_150k_10_230930.csv', 'public_up_to_150k_7_230930.csv', 'public_up_to_150k_2_230930.csv', 'public_150k_plus_230930.csv', '.ipynb_checkpoints', 'placeholder.txt', 'public_up_to_150k_8_230930.csv', 'public_up_to_150k_12_230930.csv', 'public_up_to_150k_5_230930.csv']


In [3]:
loans_1 = pd.read_csv(path + dir_list[0])
loans_2 = pd.read_csv(path + dir_list[1])
ppp = pd.concat([loans_1, loans_2])

In [4]:
ppp.shape

(1800000, 53)

In [5]:
ppp.columns

Index(['LoanNumber', 'DateApproved', 'SBAOfficeCode', 'ProcessingMethod',
       'BorrowerName', 'BorrowerAddress', 'BorrowerCity', 'BorrowerState',
       'BorrowerZip', 'LoanStatusDate', 'LoanStatus', 'Term',
       'SBAGuarantyPercentage', 'InitialApprovalAmount',
       'CurrentApprovalAmount', 'UndisbursedAmount', 'FranchiseName',
       'ServicingLenderLocationID', 'ServicingLenderName',
       'ServicingLenderAddress', 'ServicingLenderCity', 'ServicingLenderState',
       'ServicingLenderZip', 'RuralUrbanIndicator', 'HubzoneIndicator',
       'LMIIndicator', 'BusinessAgeDescription', 'ProjectCity',
       'ProjectCountyName', 'ProjectState', 'ProjectZip', 'CD', 'JobsReported',
       'NAICSCode', 'Race', 'Ethnicity', 'UTILITIES_PROCEED',
       'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED',
       'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED',
       'DEBT_INTEREST_PROCEED', 'BusinessType', 'OriginatingLenderLocationID',
       'OriginatingLender', 'Originatin

In [6]:
print(ppp['LoanStatus'].value_counts())
print(np.sum(ppp['LoanStatus'].isna())) # no nulls

LoanStatus
Paid in Full    1634401
Charged Off      111965
Exemption 4       53634
Name: count, dtype: int64
0


In [7]:
# Baseline model and class imbalance
paid_frac = np.sum(ppp['LoanStatus'] == 'Paid in Full') / ppp.shape[0]
forgiven_frac = np.sum(ppp['LoanStatus'] == 'Charged Off') / ppp.shape[0]
ex4_frac = np.sum(ppp['LoanStatus'] == 'Exemption 4') / ppp.shape[0]
print(f'Paid in full: {round(paid_frac, 3)}')
print(f'Charged Off: {round(forgiven_frac, 3)}')
print(f'Exemption 4: {round(ex4_frac, 3)}')

Paid in full: 0.908
Charged Off: 0.062
Exemption 4: 0.03


'Paid in full' is the plurality class at 0.908.  Predicting 'paid in full' 90.8% of the time represents the baseline model which a successful classifier will beat.

### Random Forest

In [8]:
# type check
ppp['SBAOfficeCode'] = ppp['SBAOfficeCode'].astype(int)

In [9]:
np.sum(ppp['LoanStatusDate'].isna()) / ppp['LoanStatusDate'].size

0.029796666666666666

We know that about 2.97% of loans are still in arrears, and the LoanStatusDate will be null. We drop this from the model features for now because this will be highly correlated with the target and lead to data leakage.  We can add this feature back in at a later iteration of the model if we want to see whether the date the loan was forgiven or repaid can predict between these two classes only.

In [10]:
np.sum(ppp['UndisbursedAmount'] > 0)

4

In [11]:
ppp[ppp['UndisbursedAmount'] > 0][['UndisbursedAmount', 'CurrentApprovalAmount']]

,UndisbursedAmount,CurrentApprovalAmount
156358,78230.00,78230.00
149720,3000.00,28400.00
772551,0.75,2262.75
809276,330000.00,0.00


Nearly all loans have been disbursed, so we drop UndisbursedAmount from the model because it will not have predictive value.

In [12]:
ppp['RuralUrbanIndicator'].value_counts()

RuralUrbanIndicator
U    1618966
R     181034
Name: count, dtype: int64

In [13]:
np.sum(ppp['RuralUrbanIndicator'].isna())

0

In [14]:
ppp['HubzoneIndicator'].value_counts()

HubzoneIndicator
N    1308199
Y     491801
Name: count, dtype: int64

In [15]:
np.sum(ppp['BusinessAgeDescription'].isna())

0

In [16]:
np.sum(ppp['ProjectCountyName'].isna())

63

We do not know what Project features refer to and there are some missing values. We elect to ignore these features.

In [17]:
np.sum(ppp['CD'].isna())

54

The congressional district associated with a loan (CD) is important for potentially connecting this dataset to related COVID public health data.  There are some missing values, which we will handle as 'unrecorded'.

In [18]:
ppp['CD'] = ppp['CD'].where(ppp['CD'].notna(), 'unrecorded')
np.sum(ppp['CD'].isna())

0

In [19]:
np.sum(ppp['JobsReported'].isna()) # All businesses report number of jobs (business size)

0

In [20]:
ppp['NAICSCode'].dtypes

dtype('float64')

In [21]:
np.sum(ppp['NAICSCode'].isna())

19483

Nearly 20,000 loans do not report the NAICSCode (type of industry) the business operates in.  This information is important for the classifier. We designate unrecorded values as '000000'.

In [22]:
# temp_ppp = ppp

In [23]:
ppp['NAICSCode'] = ppp['NAICSCode'].where(ppp['NAICSCode'].notna(), 0)
np.sum(ppp['NAICSCode'].isna())

0

In [24]:
temp = ppp['NAICSCode'].astype(int).astype(str).replace(to_replace='0', value='000000')
np.sum(temp == '000000')

19483

In [25]:
ppp['NAICSCode'] = temp

In [26]:
ppp['NAICSCode'].dtypes

dtype('O')

In [27]:
np.sum(ppp[['Race', 'Ethnicity', 'Gender', 'Veteran']].isna(), axis=0)

Race         0
Ethnicity    0
Gender       0
Veteran      0
dtype: int64

Demographic information of the business owner has no nulls.

In [28]:
np.sum(ppp[['UTILITIES_PROCEED', 'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED', 'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED', 'DEBT_INTEREST_PROCEED']].isna(), axis=0)

UTILITIES_PROCEED            1279767
PAYROLL_PROCEED                 1379
MORTGAGE_INTEREST_PROCEED    1765119
RENT_PROCEED                 1730784
REFINANCE_EIDL_PROCEED       1784389
HEALTH_CARE_PROCEED          1779194
DEBT_INTEREST_PROCEED        1784319
dtype: int64

In [29]:
# fill nulls as zero
temp = ppp[['UTILITIES_PROCEED', 'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED', 'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED', 'DEBT_INTEREST_PROCEED']]

temp = temp.replace(to_replace=np.nan, value=0.0)
np.sum(temp.isna(), axis=0)

# df = ppp.drop(['UTILITIES_PROCEED', 'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED', 'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED', 'DEBT_INTEREST_PROCEED']).merge(temp, how='left')
# df.head(10)

UTILITIES_PROCEED            0
PAYROLL_PROCEED              0
MORTGAGE_INTEREST_PROCEED    0
RENT_PROCEED                 0
REFINANCE_EIDL_PROCEED       0
HEALTH_CARE_PROCEED          0
DEBT_INTEREST_PROCEED        0
dtype: int64

In [30]:
temp_df = ppp.drop(columns=['UTILITIES_PROCEED', 'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED', 'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED', 'DEBT_INTEREST_PROCEED'])

In [31]:
temp_df['UTILITIES_PROCEED'] = temp['UTILITIES_PROCEED']

In [32]:
# temp_df.shape
# temp_df['PAYROLL_PROCEED'] = temp['PAYROLL_PROCEED']
# temp_df.shape

for proceed in ['UTILITIES_PROCEED', 'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED', 'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED', 'DEBT_INTEREST_PROCEED']:
    ppp[proceed] = temp[proceed]

ppp.shape

(1800000, 53)

I brute forced adding these columns back because the joins were just doubling the rows instead of using the shared index.

,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,LoanStatus
0,0.0,18457.00,0.0,0.0,0.0,0.0,0.0,Paid in Full
1,0.0,18457.00,0.0,0.0,0.0,0.0,0.0,Paid in Full
2,1.0,18450.65,0.0,0.0,0.0,0.0,0.0,Paid in Full
3,1.0,18455.00,0.0,0.0,0.0,0.0,0.0,Paid in Full
4,0.0,18455.00,0.0,0.0,0.0,0.0,0.0,Paid in Full
...,...,...,...,...,...,...,...,...
899995,0.0,84752.00,0.0,0.0,0.0,0.0,0.0,Paid in Full
899996,0.0,84750.00,0.0,0.0,0.0,0.0,0.0,Paid in Full
899997,0.0,63560.00,0.0,21187.0,0.0,0.0,0.0,Paid in Full
899998,1.0,84729.00,0.0,0.0,0.0,0.0,0.0,Paid in Full


In [79]:
np.sum(ppp[['BusinessType', 'OriginatingLenderLocationID']].isna(), axis=0)

BusinessType                   163
OriginatingLenderLocationID      0
dtype: int64

In [80]:
# Handle 163 null business type
bt_temp = ppp['BusinessType'].replace(to_replace=np.nan, value='unrecorded')
np.sum(bt_temp.isna())

ppp['BusinessType'] = bt_temp

In [81]:
ppp['OriginatingLenderLocationID'] = ppp['OriginatingLenderLocationID'].astype(int).astype(str)

In [82]:
# Choose features we believe will be most predictive
drop_features = ['LoanNumber', 'DateApproved', 'BorrowerName', 'BorrowerAddress', 'LoanStatusDate', 'UndisbursedAmount', 'ServicingLenderName', 'ServicingLenderAddress', 'ProjectCity', 'ProjectCountyName', 'ProjectState', 'ProjectZip', 'OriginatingLender', 'OriginatingLenderCity', 'OriginatingLenderState', 'ForgivenessAmount', 'ForgivenessDate']
temp_dropped = ppp.drop(columns=drop_features)

In [83]:
temp_dropped.columns

Index(['SBAOfficeCode', 'ProcessingMethod', 'BorrowerCity', 'BorrowerState',
       'BorrowerZip', 'LoanStatus', 'Term', 'SBAGuarantyPercentage',
       'InitialApprovalAmount', 'CurrentApprovalAmount', 'FranchiseName',
       'ServicingLenderLocationID', 'ServicingLenderCity',
       'ServicingLenderState', 'ServicingLenderZip', 'RuralUrbanIndicator',
       'HubzoneIndicator', 'LMIIndicator', 'BusinessAgeDescription', 'CD',
       'JobsReported', 'NAICSCode', 'Race', 'Ethnicity', 'UTILITIES_PROCEED',
       'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED',
       'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED',
       'DEBT_INTEREST_PROCEED', 'BusinessType', 'OriginatingLenderLocationID',
       'Gender', 'Veteran', 'NonProfit'],
      dtype='object')

In [84]:
ppp = temp_dropped

In [85]:
ppp.columns

Index(['SBAOfficeCode', 'ProcessingMethod', 'BorrowerCity', 'BorrowerState',
       'BorrowerZip', 'LoanStatus', 'Term', 'SBAGuarantyPercentage',
       'InitialApprovalAmount', 'CurrentApprovalAmount', 'FranchiseName',
       'ServicingLenderLocationID', 'ServicingLenderCity',
       'ServicingLenderState', 'ServicingLenderZip', 'RuralUrbanIndicator',
       'HubzoneIndicator', 'LMIIndicator', 'BusinessAgeDescription', 'CD',
       'JobsReported', 'NAICSCode', 'Race', 'Ethnicity', 'UTILITIES_PROCEED',
       'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED',
       'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED',
       'DEBT_INTEREST_PROCEED', 'BusinessType', 'OriginatingLenderLocationID',
       'Gender', 'Veteran', 'NonProfit'],
      dtype='object')


While these proceed data are very important, we have to decide how to best use these.  For now, we can use all of these features as-is.  We must however fill the null values as $0.00.

We fit a model to the reduced df.

In [86]:
np.sum(ppp.isna(), axis=0)

SBAOfficeCode                        0
ProcessingMethod                     0
BorrowerCity                        16
BorrowerState                        0
BorrowerZip                         13
LoanStatus                           0
Term                                 0
SBAGuarantyPercentage                0
InitialApprovalAmount                0
CurrentApprovalAmount                0
FranchiseName                  1780395
ServicingLenderLocationID            0
ServicingLenderCity                  0
ServicingLenderState                 0
ServicingLenderZip                   0
RuralUrbanIndicator                  0
HubzoneIndicator                     0
LMIIndicator                         0
BusinessAgeDescription               0
CD                                   0
JobsReported                         0
NAICSCode                            0
Race                                 0
Ethnicity                            0
UTILITIES_PROCEED                    0
PAYROLL_PROCEED          

In [87]:
ppp = ppp.drop(columns=['FranchiseName', 'NonProfit'])
np.sum(ppp.isna(), axis=0)

SBAOfficeCode                   0
ProcessingMethod                0
BorrowerCity                   16
BorrowerState                   0
BorrowerZip                    13
LoanStatus                      0
Term                            0
SBAGuarantyPercentage           0
InitialApprovalAmount           0
CurrentApprovalAmount           0
ServicingLenderLocationID       0
ServicingLenderCity             0
ServicingLenderState            0
ServicingLenderZip              0
RuralUrbanIndicator             0
HubzoneIndicator                0
LMIIndicator                    0
BusinessAgeDescription          0
CD                              0
JobsReported                    0
NAICSCode                       0
Race                            0
Ethnicity                       0
UTILITIES_PROCEED               0
PAYROLL_PROCEED                 0
MORTGAGE_INTEREST_PROCEED       0
RENT_PROCEED                    0
REFINANCE_EIDL_PROCEED          0
HEALTH_CARE_PROCEED             0
DEBT_INTEREST_

In [88]:
# drop rows with null borrower city and zip

temp = ppp[ppp['BorrowerCity'].notna()]
ppp = temp
np.sum(ppp.isna(), axis=0)

SBAOfficeCode                  0
ProcessingMethod               0
BorrowerCity                   0
BorrowerState                  0
BorrowerZip                    0
LoanStatus                     0
Term                           0
SBAGuarantyPercentage          0
InitialApprovalAmount          0
CurrentApprovalAmount          0
ServicingLenderLocationID      0
ServicingLenderCity            0
ServicingLenderState           0
ServicingLenderZip             0
RuralUrbanIndicator            0
HubzoneIndicator               0
LMIIndicator                   0
BusinessAgeDescription         0
CD                             0
JobsReported                   0
NAICSCode                      0
Race                           0
Ethnicity                      0
UTILITIES_PROCEED              0
PAYROLL_PROCEED                0
MORTGAGE_INTEREST_PROCEED      0
RENT_PROCEED                   0
REFINANCE_EIDL_PROCEED         0
HEALTH_CARE_PROCEED            0
DEBT_INTEREST_PROCEED          0
BusinessTy

Some categorical parameters have too many unique values and are also not very informative.  For example BorrowerCity, Address, Zip.
We will drop these categories as they are low priority for inclusion.

In [89]:
low_priority_cats = ['BorrowerCity', 'BorrowerState', 'ServicingLenderCity', 'ServicingLenderCity', 'ServicingLenderState', 'ServicingLenderZip', ] # 'BorrowerZip' added back for 09 model bc it is handling current feature size ok, and it may be valuable for econogeo info
ppp = ppp.drop(columns=low_priority_cats)

## Binarize the target variable to transform multi-class to binary classification
This will greatly simplify fitting and interpreting the metrics.  It also suits the premise of the business recommendations because we are interested in reducing the burden poorly allocated PPP funds, which could be redirected to other kinds of economic relief.

In [36]:
ppp['LoanStatus'] = ppp['LoanStatus'].replace({'Paid in Full' : 1, 'Charged Off' : 0, 'Exemption 4' : 0})

ppp['LoanStatus'].value_counts(normalize=True)

LoanStatus
1    0.908001
0    0.091999
Name: proportion, dtype: float64

The nature of our goal:


1 - Paid in full
0 - Defaulted

It is most important to detect loans which will be defaulted on (0). 
It is most deleterious to detect false positives. (predicting paid (1) when it is actually defaulted (0)). because that means we gave a loan which we thought would be recouped but we actually lost the money. We must maximize *precision* to prevent risk.

False negative is when we predict the loan will not be recouped (0) but we actually get the money back (1).  This is less costly because we don't lose any money directly. Although there are indirect consequences like failing to support a business that could have been saved (the opportunity cost of failing to stimulate the economy).


In [39]:
temp['LoanStatus'] = ppp['LoanStatus']
temp['CurrentApprovalAmount'] = ppp['CurrentApprovalAmount']
temp

,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,LoanStatus,CurrentApprovalAmount
0,0.0,18457.00,0.0,0.0,0.0,0.0,0.0,1,18457.00
1,0.0,18457.00,0.0,0.0,0.0,0.0,0.0,1,18457.00
2,1.0,18450.65,0.0,0.0,0.0,0.0,0.0,1,18456.65
3,1.0,18455.00,0.0,0.0,0.0,0.0,0.0,1,18456.00
4,0.0,18455.00,0.0,0.0,0.0,0.0,0.0,1,18455.00
...,...,...,...,...,...,...,...,...,...
899995,0.0,84752.00,0.0,0.0,0.0,0.0,0.0,1,84752.00
899996,0.0,84750.00,0.0,0.0,0.0,0.0,0.0,1,84750.00
899997,0.0,63560.00,0.0,21187.0,0.0,0.0,0.0,1,84747.00
899998,1.0,84729.00,0.0,0.0,0.0,0.0,0.0,1,84734.00


In [40]:
for proceed in ['UTILITIES_PROCEED', 'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED', 'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED', 'DEBT_INTEREST_PROCEED']:
    temp[proceed] = temp[proceed] / temp['CurrentApprovalAmount']

temp

,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,LoanStatus,CurrentApprovalAmount
0,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0,1,18457.00
1,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0,1,18457.00
2,0.000054,0.999675,0.0,0.000000,0.0,0.0,0.0,1,18456.65
3,0.000054,0.999946,0.0,0.000000,0.0,0.0,0.0,1,18456.00
4,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0,1,18455.00
...,...,...,...,...,...,...,...,...,...
899995,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0,1,84752.00
899996,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0,1,84750.00
899997,0.000000,0.749997,0.0,0.250003,0.0,0.0,0.0,1,84747.00
899998,0.000012,0.999941,0.0,0.000000,0.0,0.0,0.0,1,84734.00


In [44]:
# for loans that were paid off
paid_costs = temp[temp['LoanStatus'] == 1].drop(columns=['LoanStatus', 'CurrentApprovalAmount'])
paid_costs

,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED
0,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0
1,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0
2,0.000054,0.999675,0.0,0.000000,0.0,0.0,0.0
3,0.000054,0.999946,0.0,0.000000,0.0,0.0,0.0
4,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...
899995,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0
899996,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0
899997,0.000000,0.749997,0.0,0.250003,0.0,0.0,0.0
899998,0.000012,0.999941,0.0,0.000000,0.0,0.0,0.0


In [60]:
avg_df = temp.groupby('LoanStatus').mean()
avg_df

,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,CurrentApprovalAmount
LoanStatus,,,,,,,,
0,inf,inf,0.000442,0.002417,0.000448,0.000119,0.000148,19333.679308
1,0.006987,0.98336,0.002031,0.006086,0.000457,0.000713,0.000307,25010.915008


In [63]:
avg_df.columns

Index(['UTILITIES_PROCEED', 'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED',
       'RENT_PROCEED', 'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED',
       'DEBT_INTEREST_PROCEED', 'CurrentApprovalAmount'],
      dtype='object')

In [70]:
avg_df.iloc[1:, :].values

array([[6.98662414e-03, 9.83359520e-01, 2.03052927e-03, 6.08584644e-03,
        4.56917565e-04, 7.12586513e-04, 3.06511642e-04, 2.50109150e+04]])

In [72]:
plt.bar(avg_df.iloc[1:, :].values)

TypeError: bar() missing 1 required positional argument: 'height'

In [73]:
plt.bar(data=avg_df.iloc[1:, :])

TypeError: bar() missing 2 required positional arguments: 'x' and 'height'

In [74]:
type(avg_df.iloc[1:, :])

pandas.core.frame.DataFrame

In [75]:
ppp.groupby('LoanStatus')['SBAGuarantyPercentage'].mean()

LoanStatus
0    100.0
1    100.0
Name: SBAGuarantyPercentage, dtype: float64

In [79]:
terms = ppp.groupby('LoanStatus')['Term'].mean()

In [81]:
terms

LoanStatus
0    40.927898
1    46.697430
Name: Term, dtype: float64

TypeError: bar() missing 1 required positional argument: 'height'

## Here
A reduced, cleaned df that is ready to be fit to a rf model.

In [91]:
ppp.to_csv('../data/cleaned_1and2_reduced.csv', index=False)